In [1]:
import pandas as pd
import sys
sys.path.append('/home/tomek/ib_tools')
from datastore import ArcticStore
from indicators import get_ATR
from grouper import group_by_volume

/home/tomek/.virtualenvs/ib/lib/python3.7/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
store = ArcticStore('TRADES_30_secs')
symbols = ['NQ', 'GC', 'ES', 'YM']
volumes = {'NQ': 10000,
         'GC': 5500,
         'ES': 43000,
         'YM': 8000}

Library created, but couldn't enable sharding: no such command: 'enablesharding'. This is OK if you're not 'admin'


In [3]:
dfs = {symbol: store.contfuture(symbol, index=-2, start_date='20190101', end_date='20191231') for symbol in symbols}
dfs

{'NQ':                         open     high      low    close  volume   average  \
 date                                                                        
 2019-01-02 00:00:00  6349.50  6355.75  6344.00  6347.75     243  6348.650   
 2019-01-02 00:00:30  6348.50  6353.00  6347.50  6353.00      89  6349.750   
 2019-01-02 00:01:00  6352.50  6364.75  6351.50  6359.25     178  6358.450   
 2019-01-02 00:01:30  6359.50  6361.75  6358.50  6361.25     118  6360.175   
 2019-01-02 00:02:00  6360.75  6363.00  6359.25  6362.50      49  6361.325   
 ...                      ...      ...      ...      ...     ...       ...   
 2019-12-30 22:58:00  8732.50  8732.50  8732.00  8732.25      13  8732.250   
 2019-12-30 22:58:30  8732.25  8732.25  8731.75  8732.25      11  8732.075   
 2019-12-30 22:59:00  8732.25  8732.25  8730.75  8731.75      38  8731.400   
 2019-12-30 22:59:30  8732.00  8732.50  8731.75  8732.50      21  8732.225   
 2019-12-31 00:00:00  8731.75  8732.00  8727.25  8730.25  

In [4]:
candles = {symbol: group_by_volume(df, volumes[symbol]) for symbol, df in dfs.items()}
candles

{'NQ':                         open     high      low    close  barCount  volume
 date                                                                     
 2019-01-02 02:12:00  6349.50  6386.25  6344.00  6382.75      6738   10062
 2019-01-02 03:21:30  6382.75  6384.75  6320.00  6325.00      7427   10078
 2019-01-02 04:43:30  6324.50  6327.00  6292.25  6306.25      7464   10020
 2019-01-02 06:39:00  6306.50  6307.50  6258.75  6277.25      7293   10004
 2019-01-02 08:33:00  6277.25  6277.25  6247.00  6257.25      7253   10032
 ...                      ...      ...      ...      ...       ...     ...
 2019-12-30 21:23:00  8735.50  8740.75  8729.25  8739.50      5040   10087
 2019-12-30 21:44:00  8739.50  8742.75  8731.00  8733.25      4952   10206
 2019-12-30 21:56:30  8733.25  8735.50  8724.25  8729.00      5067   10460
 2019-12-30 22:00:00  8728.75  8737.75  8720.25  8735.00      4285   10498
 2019-12-31 00:00:00  8735.00  8738.00  8727.25  8730.25      3947    7826
 
 [10458 rows x 6 

In [5]:
atrs = {symbol: get_ATR(df, 180) for symbol, df in candles.items()}
atrs

{'NQ': date
 2019-01-02 02:12:00    42.250000
 2019-01-02 03:21:30    53.562500
 2019-01-02 04:43:30    47.221862
 2019-01-02 06:39:00    47.610288
 2019-01-02 08:33:00    44.060647
                          ...    
 2019-12-30 21:23:00    12.603672
 2019-12-30 21:44:00    12.594239
 2019-12-30 21:56:30    12.579385
 2019-12-30 22:00:00    12.633757
 2019-12-31 00:00:00    12.612942
 Name: ATR, Length: 10458, dtype: float64, 'GC': date
 2019-01-02 02:06:00    3.900000
 2019-01-02 03:04:30    2.743611
 2019-01-02 04:00:00    2.863730
 2019-01-02 05:42:30    2.542512
 2019-01-02 06:36:30    2.513373
                          ...   
 2019-12-30 18:09:00    1.809510
 2019-12-30 18:57:00    1.800566
 2019-12-30 19:38:00    1.791720
 2019-12-30 21:04:00    1.786286
 2019-12-31 00:00:00    1.778703
 Name: ATR, Length: 12137, dtype: float64, 'ES': date
 2019-01-02 03:18:30    25.250000
 2019-01-02 06:57:00    23.113194
 2019-01-02 09:21:30    25.097306
 2019-01-02 10:26:30    22.340116
 2019-0

In [6]:
out = {}
out1 = {}
out2 = {}
for symbol, series in atrs.items():
    print(symbol)
    data = series.describe()
    print(data)
    out[symbol] = round(data['25%'],1)
    out1[symbol] = round(data['mean'] - 2*data['std'], 1)
    out2[symbol] = round(data['50%'],1)

NQ
count    10458.000000
mean        17.393009
std          3.687095
min         10.482281
25%         14.622140
50%         16.989944
75%         19.384188
max         53.562500
Name: ATR, dtype: float64
GC
count    12137.000000
mean         2.319752
std          0.520703
min          1.487171
25%          1.867842
50%          2.193399
75%          2.765625
max          3.900000
Name: ATR, dtype: float64
ES
count    7654.000000
mean        6.083668
std         1.454923
min         3.901333
25%         5.008710
50%         5.921129
75%         6.753479
max        25.250000
Name: ATR, dtype: float64
YM
count    5828.000000
mean       64.076346
std        12.988836
min        44.721147
25%        54.333403
50%        61.744539
75%        70.370741
max       161.183333
Name: ATR, dtype: float64


In [7]:
out

{'NQ': 14.6, 'GC': 1.9, 'ES': 5.0, 'YM': 54.3}

In [8]:
out1

{'NQ': 10.0, 'GC': 1.3, 'ES': 3.2, 'YM': 38.1}

In [9]:
out2

{'NQ': 17.0, 'GC': 2.2, 'ES': 5.9, 'YM': 61.7}